# Working with Open Model Zoo Models

> **Note:** This notebook has been moved to a new branch named "latest". [Click here](https://github.com/openvinotoolkit/openvino_notebooks/blob/latest/notebooks/model-tools/model-tools.ipynb) to get the most updated version of the notebook. This branch is deprecated.
This tutorial shows how to download a model from [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo), convert it to OpenVINO™ IR format, show information about the model, and benchmark the model.


#### Table of contents:

- [OpenVINO and Open Model Zoo Tools](#OpenVINO-and-Open-Model-Zoo-Tools)
- [Preparation](#Preparation)
    - [Model Name](#Model-Name)
    - [Imports](#Imports)
    - [Settings and Configuration](#Settings-and-Configuration)
- [Download a Model from Open Model Zoo](#Download-a-Model-from-Open-Model-Zoo)
- [Convert a Model to OpenVINO IR format](#Convert-a-Model-to-OpenVINO-IR-format)
- [Get Model Information](#Get-Model-Information)
- [Run Benchmark Tool](#Run-Benchmark-Tool)
    - [Benchmark with Different Settings](#Benchmark-with-Different-Settings)



## OpenVINO and Open Model Zoo Tools
[back to top ⬆️](#Table-of-contents:)

OpenVINO and Open Model Zoo tools are listed in the table below.

| Tool             | Command             | Description                                             |
|:-----------------|:--------------------|:--------------------------------------------------------|
| Model Downloader | `omz_downloader`      | Download models from Open Model Zoo.                    |
| Model Converter  | `omz_converter`       | Convert Open Model Zoo models to OpenVINO's IR format.  |
| Info Dumper      | `omz_info_dumper`     | Print information about Open Model Zoo models.          |
| Benchmark Tool   | `benchmark_app`       | Benchmark model performance by computing inference time.|

In [ ]:
# Install openvino package
%pip install -q "openvino-dev>=2024.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 9.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch

## Preparation
[back to top ⬆️](#Table-of-contents:)

### Model Name
[back to top ⬆️](#Table-of-contents:)

Set `model_name` to the name of the Open Model Zoo model to use in this notebook. Refer to the list of [public](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/public/index.md) and [Intel](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/intel/index.md) pre-trained models for a full list of models that can be used. Set `model_name` to the model you want to use.

In [ ]:
# model_name = "resnet-50-pytorch"
model_name = "mobilenet-v2-pytorch"

In [ ]:
!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 52.4 MB/s eta 0:00:00


### Imports
[back to top ⬆️](#Table-of-contents:)


In [ ]:
import json
from pathlib import Path

import openvino as ov
from IPython.display import Markdown, display

# Fetch `notebook_utils` module
import urllib.request
urllib.request.urlretrieve(
    url='https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/main/notebooks/utils/notebook_utils.py',
    filename='notebook_utils.py'
)
from notebook_utils import DeviceNotFoundAlert, NotebookAlert

### Settings and Configuration
[back to top ⬆️](#Table-of-contents:)

Set the file and directory paths. By default, this notebook downloads models from Open Model Zoo to the `open_model_zoo_models` directory in your `$HOME` directory. On Windows, the $HOME directory is usually `c:\users\username`, on Linux `/home/username`. To change the folder, change `base_model_dir` in the cell below.

The following settings can be changed:

* `base_model_dir`: Models will be downloaded into the `intel` and `public` folders in this directory.
* `omz_cache_dir`: Cache folder for Open Model Zoo. Specifying a cache directory is not required for Model Downloader and Model Converter, but it speeds up subsequent downloads.
* `precision`: If specified, only models with this precision will be downloaded and converted.

In [ ]:
base_model_dir = Path("model")
omz_cache_dir = Path("cache")
precision = "FP16"

# Check if an GPU is available on this system to use with Benchmark App.
core = ov.Core()
gpu_available = "GPU" in core.available_devices

print(
    f"base_model_dir: {base_model_dir}, omz_cache_dir: {omz_cache_dir}, gpu_availble: {gpu_available}"
)

base_model_dir: model, omz_cache_dir: cache, gpu_availble: False


## Download a Model from Open Model Zoo
[back to top ⬆️](#Table-of-contents:)



Specify, display and run the Model Downloader command to download the model.

In [ ]:
## Uncomment the next line to show help in omz_downloader which explains the command-line options.

# !omz_downloader --help

$download_command 명령어는 OpenVINO™ Toolkit의 일부로 제공되는 Model Downloader (omz_downloader)를 사용하고 있습니다.

In [ ]:
download_command = (
    f"omz_downloader --name {model_name} --output_dir {base_model_dir} --cache_dir {omz_cache_dir}"
)
display(Markdown(f"Download command: `{download_command}`"))
display(Markdown(f"Downloading {model_name}..."))
! $download_command

Download command: `omz_downloader --name mobilenet-v2-pytorch --output_dir model --cache_dir cache`

Downloading mobilenet-v2-pytorch...

################|| Downloading mobilenet-v2-pytorch ||################

========== Retrieving model/public/mobilenet-v2-pytorch/mobilenet_v2-b0353104.pth from the cache



## Convert a Model to OpenVINO IR format
[back to top ⬆️](#Table-of-contents:)

Specify, display and run the Model Converter command to convert the model to OpenVINO IR format. Model conversion may take a while. The output of the Model Converter command will be displayed. When the conversion is successful, the last lines of the output will include: `[ SUCCESS ] Generated IR version 11 model.` For downloaded models that are already in OpenVINO IR format, conversion will be skipped.

In [ ]:
## Uncomment the next line to show Help in omz_converter which explains the command-line options.

# !omz_converter --help

PyTorch에서 구현된 MobileNetV2 모델을 ONNX(Open Neural Network Exchange) 형식으로 변환하는 것은 여러 가지 면에서 의미가 있습니다. ONNX는 다양한 플랫폼과 프레임워크 간의 모델 호환성을 제공하여, 다양한 환경에서 모델을 쉽게 배포하고 실행할 수 있게 해줍니다. MobileNetV2를 ONNX로 변환하는 것이 가지는 주요 이점을 설명드리겠습니다:

1. 플랫폼 및 프레임워크 독립성

ONNX는 모델을 다양한 머신러닝 프레임워크에서 사용할 수 있도록 지원합니다. 예를 들어, PyTorch에서 훈련된 모델을 ONNX로 변환하면, TensorFlow, Microsoft Cognitive Toolkit (CNTK), Apple CoreML 등 다른 프레임워크에서도 사용할 수 있습니다. 이는 개발자가 모델을 더 넓은 범위의 애플리케이션과 시스템에 적용할 수 있게 합니다.

2. 배포 용이성

ONNX 모델은 클라우드, 모바일, 임베디드 시스템과 같이 다양한 환경에서 실행될 수 있습니다. 이는 특히 IoT 디바이스나 모바일 애플리케이션 같이 컴퓨팅 자원이 제한된 환경에서 모델을 배포할 때 큰 장점이 됩니다.

3. 성능 최적화

ONNX는 하드웨어 가속화를 지원하는 다양한 최적화를 제공할 수 있습니다. ONNX 모델은 특정 하드웨어(예: GPU, FPGA)에서 실행할 때 성능을 최적화하기 위해 다양한 도구와 라이브러리를 사용할 수 있습니다. 이를 통해 실행 속도와 효율성이 크게 향상될 수 있습니다.

4. 유지 관리 및 업데이트의 용이성

ONNX 형식으로 모델을 표준화하면, 모델의 버전 관리와 업데이트가 더욱 용이해집니다. 모델을 쉽게 업데이트하고 배포할 수 있으며, 호환성 문제 없이 다양한 환경에서 일관된 성능을 제공할 수 있습니다.

In [ ]:
convert_command = f"omz_converter --name {model_name} --precisions {precision} --download_dir {base_model_dir} --output_dir {base_model_dir}"
display(Markdown(f"Convert command: `{convert_command}`"))
display(Markdown(f"Converting {model_name}..."))

! $convert_command

Convert command: `omz_converter --name mobilenet-v2-pytorch --precisions FP16 --download_dir model --output_dir model`

Converting mobilenet-v2-pytorch...

========== Converting mobilenet-v2-pytorch to ONNX
Conversion to ONNX command: /usr/bin/python3 -- /usr/local/lib/python3.10/dist-packages/omz_tools/internal_scripts/pytorch_to_onnx.py --model-name=mobilenet_v2 --weights=model/public/mobilenet-v2-pytorch/mobilenet_v2-b0353104.pth --import-module=torchvision.models --input-shape=1,3,224,224 --output-file=model/public/mobilenet-v2-pytorch/mobilenet-v2.onnx --input-names=data --output-names=prob

ONNX check passed successfully.

========== Converting mobilenet-v2-pytorch to IR (FP16)
Conversion command: /usr/bin/python3 -- /usr/local/bin/mo --framework=onnx --output_dir=model/public/mobilenet-v2-pytorch/FP16 --model_name=mobilenet-v2-pytorch --input=data '--mean_values=data[123.675,116.28,103.53]' '--scale_values=data[58.624,57.12,57.375]' --reverse_input_channels --output=prob --input_model=model/public/mobilenet-v2-pytorch/mobilenet-v2.onnx '--layout=data(NCHW)' '--input_shape=[1, 3, 224, 224]' --compress_to_fp16=True

[ INFO ] Generate

## Get Model Information
[back to top ⬆️](#Table-of-contents:)

The Info Dumper prints the following information for Open Model Zoo models:

* Model name
* Description
* Framework that was used to train the model
* License URL
* Precisions supported by the model
* Subdirectory: the location of the downloaded model
* Task type

This information can be shown by running `omz_info_dumper --name model_name` in a terminal. The information can also be parsed and used in scripts.

In the next cell, run Info Dumper and use `json` to load the information in a dictionary.

json.loads():

json 모듈은 Python에서 JSON 데이터를 다룰 때 사용합니다.
loads() 함수는 JSON 형식의 문자열을 입력으로 받아, 그 내용을 Python의 데이터 타입(예: 딕셔너리, 리스트)으로 변환합니다. loads는 "load string"의 줄임말입니다.

model_info_output.get_nlstr():

이 부분은 model_info_output 객체의 get_nlstr() 메서드를 호출하는 것으로 보입니다. get_nlstr() 메서드의 정확한 기능은 model_info_output 객체의 정의에 따라 다르지만, 일반적으로 객체에서 특정 데이터를 문자열 형태로 추출하는 메서드일 가능성이 높습니다.
여기서 get_nlstr()는 JSON 형식의 문자열을 반환할 것으로 예상됩니다.

In [ ]:
model_info_output = %sx omz_info_dumper --name $model_name
model_info = json.loads(model_info_output.get_nlstr())

if len(model_info) > 1:
    NotebookAlert(
        f"There are multiple IR files for the {model_name} model. The first model in the "
        "omz_info_dumper output will be used for benchmarking. Change "
        "`selected_model_info` in the cell below to select a different model from the list.",
        "warning",
    )

model_info

[{'name': 'mobilenet-v2-pytorch',
  'composite_model_name': None,
  'description': 'MobileNet V2 is image classification model pre-trained on ImageNet dataset. This is a PyTorch* implementation of MobileNetV2 architecture as described in the paper "Inverted Residuals and Linear Bottlenecks: Mobile Networks for Classification, Detection and Segmentation" <https://arxiv.org/abs/1801.04381>.\nThe model input is a blob that consists of a single image of "1, 3, 224, 224" in "RGB" order.\nThe model output is typical object classifier for the 1000 different classifications matching with those in the ImageNet database.',
  'framework': 'pytorch',
  'license_url': 'https://raw.githubusercontent.com/pytorch/vision/master/LICENSE',
  'accuracy_config': '/usr/local/lib/python3.10/dist-packages/omz_tools/models/public/mobilenet-v2-pytorch/accuracy-check.yml',
  'model_config': '/usr/local/lib/python3.10/dist-packages/omz_tools/models/public/mobilenet-v2-pytorch/model.yml',
  'precisions': ['FP16', 

Having information of the model in a JSON file enables extraction of the path to the model directory, and building the path to the OpenVINO IR file.

In [ ]:
selected_model_info = model_info[0]
model_path = (
    base_model_dir
    / Path(selected_model_info["subdirectory"])
    / Path(f"{precision}/{selected_model_info['name']}.xml")
)
print(model_path, "exists:", model_path.exists())

model/public/mobilenet-v2-pytorch/FP16/mobilenet-v2-pytorch.xml exists: True


## Run Benchmark Tool
[back to top ⬆️](#Table-of-contents:)

By default, Benchmark Tool runs inference for 60 seconds in asynchronous mode on CPU. It returns inference speed as latency (milliseconds per image) and throughput values (frames per second).

In [ ]:
## Uncomment the next line to show Help in benchmark_app which explains the command-line options.
# !benchmark_app --help

In [ ]:
benchmark_command = f"benchmark_app -m {model_path} -t 15"
display(Markdown(f"Benchmark command: `{benchmark_command}`"))
display(Markdown(f"Benchmarking {model_name} on CPU with async inference for 15 seconds..."))

! $benchmark_command

Benchmark command: `benchmark_app -m model/public/mobilenet-v2-pytorch/FP16/mobilenet-v2-pytorch.xml -t 15`

Benchmarking mobilenet-v2-pytorch on CPU with async inference for 15 seconds...

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2024.1.0-15008-f4afc983258-releases/2024/1
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2024.1.0-15008-f4afc983258-releases/2024/1
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[ WARNING ] Performance hint was not explicitly specified in command line. Device(CPU) performance hint will be set to PerformanceMode.THROUGHPUT.
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 18.20 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     data (node: data) : f32 / [N,C,H,W] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     prob (node: prob) : f32 / [...] / [1,1000]
[Step 5/11] Resizing model to match image sizes and given batch
[ INFO ] Model batch size: 1
[Step 6/11] Configuring in